# Clustering Crypto

In [189]:
# Initial imports
import requests
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [190]:
print(f"Current working directory:{Path.cwd()}")

Current working directory:/Users/aalfiler/Sites/hw_AWS/AWS/Code


In [191]:
# Alternatively, use the provided csv file:
file_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [192]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=["Unnamed: 0"])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [193]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [194]:
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144 entries, 0 to 1247
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [195]:
IsTrading_check = crypto_df['IsTrading'].unique()
IsTrading_check

array([ True])

In [196]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [197]:
crypto_df.shape

(1144, 5)

In [198]:
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [199]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna().copy()

In [200]:
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [201]:
crypto_df.shape

(685, 5)

In [202]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [203]:
# Drop rows where there are 'N/A' text values --- need verify
crypto_df.loc[~(crypto_df=='N/A').all(axis=1)]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [204]:
crypto_df = crypto_df.reset_index()
crypto_df.head()

,index,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
2,5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
3,7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
4,8,Ethereum,Ethash,PoW,1.076842e+08,0


In [205]:
crypto_df.drop(columns=['index'], inplace=True)

In [206]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df.drop(columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"])
CoinName_df.head()

,CoinName
0,42 Coin
1,404Coin
2,EliteCoin
3,Bitcoin
4,Ethereum


In [207]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df2 = crypto_df.drop(columns="CoinName")
crypto_df2

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
1,Scrypt,PoW/PoS,1.055185e+09,532000000
2,X13,PoW/PoS,2.927942e+10,314159265359
3,SHA-256,PoW,1.792718e+07,21000000
4,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
528,SHA-256,DPoS,2.000000e+09,2000000000
529,Scrypt,PoW/PoS,1.493105e+07,250000000
530,CryptoNight,PoW,9.802226e+08,1400222610
531,Equihash,PoW,7.296538e+06,21000000


In [208]:
# Create dummy variables for text features
crypto_df2_encoded = pd.get_dummies(crypto_df2, columns=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"])
crypto_df2_encoded.head()

,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
# Standardize data
data_scaler = StandardScaler()

In [210]:
data_scaler.fit(crypto_df2_encoded)

StandardScaler()

In [211]:
crypto_df2_scaled = data_scaler.transform(crypto_df2_encoded)
crypto_df2_scaled[:5]

array([[-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555]])

In [212]:
crypto_df2_path = Path("/Users/aalfiler/Sites/hw_AWS/AWS/Code/Resources/crypto_data_encoded.csv") # store scaled data instead?
crypto_df2_encoded.to_csv(crypto_df2_path, index=False)

### Reducing Dimensions Using PCA

In [213]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_df2_pca = pca.fit_transform(crypto_df2_scaled)

In [214]:
# Create a DataFrame with the principal components data
crypto_df2_pca = pd.DataFrame(
    data=crypto_df2_pca, columns=["principal component 1", "principal component 2", 'principal component 3']
)
crypto_df2_pca.head()

,principal component 1,principal component 2,principal component 3
0,0.089017,0.318226,-0.001783
1,-0.081416,0.314342,0.142036
2,-0.425015,-0.087739,0.037902
3,0.138615,-0.183220,-0.084287
4,-0.104501,-0.048928,-0.271306


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [215]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df2_pca)
    inertia.append(km.inertia_)

In [216]:
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [217]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(crypto_df2_pca)
# Predict clusters
predictions = model.predict(crypto_df2_pca)

crypto_df2_pca["class"] = model.labels_

# Create a new DataFrame including predicted clusters and cryptocurrencies features -- ERRORRRRRRRRR
clustered_df = pd.concat([crypto_df2, crypto_df2_pca, CoinName_df], axis='columns', join='inner')
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
0,Scrypt,PoW/PoS,4.199995e+01,42,0.089017,0.318226,-0.001783,0,42 Coin
1,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.081416,0.314342,0.142036,0,404Coin
2,X13,PoW/PoS,2.927942e+10,314159265359,-0.425015,-0.087739,0.037902,0,EliteCoin
3,SHA-256,PoW,1.792718e+07,21000000,0.138615,-0.183220,-0.084287,0,Bitcoin
4,Ethash,PoW,1.076842e+08,0,-0.104501,-0.048928,-0.271306,0,Ethereum


In [218]:
clustered_df.set_index(clustered_df['CoinName'], inplace=True)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
CoinName,,,,,,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,0.089017,0.318226,-0.001783,0,42 Coin
404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.081416,0.314342,0.142036,0,404Coin
EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,-0.425015,-0.087739,0.037902,0,EliteCoin
Bitcoin,SHA-256,PoW,1.792718e+07,21000000,0.138615,-0.183220,-0.084287,0,Bitcoin
Ethereum,Ethash,PoW,1.076842e+08,0,-0.104501,-0.048928,-0.271306,0,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000,0.120436,0.030179,-1.180945,0,ZEPHYR
Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000,0.002624,0.232988,0.122527,0,Gapcoin
Beldex,CryptoNight,PoW,9.802226e+08,1400222610,-0.026734,-0.208385,0.032581,0,Beldex


### Visualizing Results

#### 3D-Scatter with Clusters

In [219]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_df2_pca,
    x="principal component 3",
    y="principal component 2",
    z="principal component 1",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [237]:
# Table with tradable cryptos
tradable_cryptos = pd.DataFrame(data=clustered_df, columns=['TotalCoinsMined', 'TotalCoinSupply'])
tradable_cryptos

,TotalCoinsMined,TotalCoinSupply
CoinName,,
42 Coin,4.199995e+01,42
404Coin,1.055185e+09,532000000
EliteCoin,2.927942e+10,314159265359
Bitcoin,1.792718e+07,21000000
Ethereum,1.076842e+08,0
...,...,...
ZEPHYR,2.000000e+09,2000000000
Gapcoin,1.493105e+07,250000000
Beldex,9.802226e+08,1400222610


In [238]:
# Print the total number of tradable cryptocurrencies
tradable_cryptos.nunique()

TotalCoinsMined    519
TotalCoinSupply    280
dtype: int64

#### Scatter Plot with Tradable Cryptocurrencies

In [239]:
# Scale data to create the scatter plot
tradable_cryptos.describe()

,TotalCoinsMined
count,5.330000e+02
mean,5.319333e+09
std,4.560537e+10
min,-5.917978e+09
25%,8.305775e+06
50%,2.448794e+07
75%,1.835340e+08
max,9.899887e+11


In [240]:
data_scaler = StandardScaler()

In [241]:
data_scaler.fit(tradable_cryptos)

StandardScaler()

In [242]:
tradable_cryptos = data_scaler.transform(tradable_cryptos)
tradable_cryptos[:5]

array([[-0.11674788, -0.15286468],
       [-0.09358885, -0.14499604],
       [ 0.52587231,  4.4937636 ],
       [-0.11635442, -0.15255408],
       [-0.11438445, -0.15286468]])

In [247]:
tradable_cryptos = pd.DataFrame(
    data=tradable_cryptos)
tradable_cryptos.head()

,0,1
0,-0.116748,-0.152865
1,-0.093589,-0.144996
2,0.525872,4.493764
3,-0.116354,-0.152554
4,-0.114384,-0.152865


In [245]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
tradable_cryptos.hvplot.scatter(
    title="Crypto Supply",
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    xlabel="Coin Mined", 
    ylabel="Coin Supply", 
    hover_cols=["CoinName"], 
    width=1000,
    height=500,
)

AttributeError: 'numpy.ndarray' object has no attribute 'plot'